In [1]:
import math
import numpy as np
from sklearn.cluster import KMeans
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [2]:
# --------------------------------------
# Step 1: 입력 데이터 정의 (지점 좌표)
# --------------------------------------
points = [
    {"id": 1, "latitude": 37.5665, "longitude": 126.9780},  # 서울
    {"id": 2, "latitude": 37.4563, "longitude": 126.7052},  # 인천
    {"id": 3, "latitude": 37.3850, "longitude": 126.9021},  # 안양
    # ... 실제 사용 시 전체 지점 추가
]

In [3]:
# --------------------------------------
# Step 2.5: 필요 트럭 수 계산
# --------------------------------------
def required_trucks(total_items, capacity_per_truck=50):
    return math.ceil(total_items / capacity_per_truck)

total_items = len(points)
max_per_vehicle = 50
num_vehicles = required_trucks(total_items, max_per_vehicle)
print(f"총 지점 수: {total_items}, 필요 차량 수: {num_vehicles}")


총 지점 수: 3, 필요 차량 수: 1


In [4]:
# Step 3: 지점 클러스터링 (KMeans)
# --------------------------------------
coords = np.array([[p["latitude"], p["longitude"]] for p in points])
kmeans = KMeans(n_clusters=num_vehicles, random_state=42)
labels = kmeans.fit_predict(coords)

# 클러스터별 분리
clustered_points = [[] for _ in range(num_vehicles)]
for point, label in zip(points, labels):
    clustered_points[label].append(point)

In [5]:
# --------------------------------------
# Step 4: 각 클러스터별 CVRP 최적화
# --------------------------------------
def compute_euclidean_distance_matrix(locations):
    size = len(locations)
    matrix = {}
    for from_idx in range(size):
        matrix[from_idx] = {}
        for to_idx in range(size):
            if from_idx == to_idx:
                matrix[from_idx][to_idx] = 0
            else:
                dx = locations[from_idx][0] - locations[to_idx][0]
                dy = locations[from_idx][1] - locations[to_idx][1]
                matrix[from_idx][to_idx] = int(math.hypot(dx, dy) * 1e6)
    return matrix

def solve_cvrp(cluster, vehicle_capacity=50):
    # 좌표 및 수요 정의
    depot = cluster[0]  # 첫 지점을 depot으로 사용
    locations = [[p["latitude"], p["longitude"]] for p in cluster]
    demands = [1] * len(cluster)
    distance_matrix = compute_euclidean_distance_matrix(locations)

    # OR-Tools 데이터 모델
    data = {
        "distance_matrix": distance_matrix,
        "demands": demands,
        "vehicle_capacities": [vehicle_capacity],
        "num_vehicles": 1,
        "depot": 0,
    }

    # Routing Index Manager & Model
    manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]), data["num_vehicles"], data["depot"])
    routing = pywrapcp.RoutingModel(manager)

    # 거리 함수 설정
    def distance_callback(from_index, to_index):
        return data["distance_matrix"][manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # 용량 제약
    def demand_callback(from_index):
        return data["demands"][manager.IndexToNode(from_index)]
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,
        data["vehicle_capacities"],
        True,
        "Capacity"
    )

    # Search parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    # Solve
    solution = routing.SolveWithParameters(search_parameters)

    # 결과 출력
    if solution:
        index = routing.Start(0)
        route = []
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
        print(f"🚚 최적 경로: {route}")
    else:
        print("❌ 경로를 찾을 수 없습니다.")

In [6]:
# --------------------------------------
# Step 5: 클러스터별 경로 최적화 실행
# --------------------------------------
for i, cluster in enumerate(clustered_points):
    print(f"\n🔹 클러스터 {i+1} (지점 수: {len(cluster)}개)")
    solve_cvrp(cluster, vehicle_capacity=max_per_vehicle)


🔹 클러스터 1 (지점 수: 3개)
🚚 최적 경로: [0, 2, 1, 0]
